In [1]:
import torch
import numpy as np

from datasets import VOCDectectionDataset
from sklearn.metrics import average_precision_score
from models import *
from utils import *
from config import *
from tqdm.notebook import tqdm

In [2]:
# propose_way = "edge_box"
year='2007'
pretrained='vgg16'
if pretrained == 'alexnet':
    model = Combined_Alexnet()
elif pretrained == 'vgg16':
    model = Combined_VGG16()
model.to(cfg.DEVICE)

checkpoints = torch.load(cfg.PATH.PT_PATH + "BestModel_2007_vgg16_26.pt")
model.load_state_dict(checkpoints['whole_model_state_dict'])


traindata = VOCDectectionDataset("~/data/", year, 'trainval', use_corloc=True)
traindata_loader = data.DataLoader(traindata, 1, shuffle=False)


In [ ]:
positives = np.array([0 for _ in range(20)]) # 
hits = np.array([0 for _ in range(20)]) # 
n = 0

with torch.no_grad():
    model.eval();
    for n_imgs, gt, n_regions, region in tqdm(traindata_loader, "Evaluation"):
        n += 1
        region = region.to(cfg.DEVICE)
        avg_scores = torch.zeros((len(region[0]), 20), dtype=torch.float32)
        scales = len(cfg.DATA.SCALES)
        k = cfg.K
        for i in range(2 * scales):
            per_img = n_imgs[i].to(cfg.DEVICE)
            per_region = n_regions[i].to(cfg.DEVICE)
            ref_scores1, ref_scores2, ref_scores3, proposal_scores = model(per_img, per_region)
            avg_scores += (ref_scores1 + ref_scores2 + ref_scores3)[:, :-1].detach().cpu() / k
        avg_scores /= 2 * scales
        
        gt = gt[0]
        gt_box = gt[:, :4]
        gt_box = gt_box.to(cfg.DEVICE)
    
    
        target = [0 for _ in range(len(VOC_CLASSES))]
        gt_target = gt[:, -1]
        for t in gt_target:
            target[int(t)] = 1.0
        gt_target = torch.Tensor(target).to(cfg.DEVICE)
        
        region = region[0].cpu()
        
        for c in range(20):
            if gt_target[c] == 0.0:
                continue
            else:
                positives[c] += 1
                cls_region = region
                cls_scores = avg_scores[:, c]
                
#                 nms_filter = nms(cls_region, cls_scores, 0.3)
#                 cls_region = cls_region[nms_filter]
#                 cls_scores = cls_scores[nms_filter]
                
                most_confident_one = torch.argmax(cls_scores)
                most_confident_region = region[most_confident_one:most_confident_one+1].to(cfg.DEVICE)
#                 most_confident_region = region[nms_filter][most_confident_one:most_confident_one+1]
                IOU = one2allbox_iou(most_confident_region, gt_box)
                if (IOU >= 0.5).sum() >= 1.0:
                    hits[c] += 1
#         if n == 3:
#             break
        if n % 500 == 0:
            print(f"CorLoc is {hits/positives}")
    corloc = hits/positives
    print(f"CorLoc is {corloc}")
    print(f"mean CorLoc is {(corloc).sum()/20}")

CorLoc is [0.88461538 0.84615385 0.56666667 0.34782609 0.23809524 0.94117647
 0.87804878 0.4137931  0.31372549 0.61111111 0.4        0.6
 0.48148148 0.90909091 0.2287234  0.52631579 0.81818182 0.45454545
 0.53571429 0.39130435]
CorLoc is [0.86       0.77358491 0.60294118 0.39473684 0.42553191 0.78571429
 0.88275862 0.41538462 0.29473684 0.61290323 0.36956522 0.62352941
 0.46296296 0.9        0.21       0.34210526 0.66666667 0.55319149
 0.5        0.31481481]
CorLoc is [0.8358209  0.80263158 0.62135922 0.35714286 0.47222222 0.76595745
 0.8708134  0.39795918 0.30075188 0.7        0.38461538 0.5625
 0.45121951 0.89333333 0.2027027  0.38095238 0.65714286 0.57575758
 0.52777778 0.32876712]
CorLoc is [0.84883721 0.79       0.63013699 0.35616438 0.43877551 0.74242424
 0.86397059 0.43076923 0.30681818 0.70175439 0.41463415 0.51724138
 0.45132743 0.88659794 0.1959799  0.34020619 0.59090909 0.5625
 0.57843137 0.30693069]
CorLoc is [0.84955752 0.80487805 0.60674157 0.36666667 0.42063492 0.7261904